In [55]:
class Doubleton(type):
    
    instances = []
    
    def __call__(cls, *args, **kwargs):# Call is called when instance is created vs New is called before class is created
        print("in call")
        if len(cls.instances)< 2:
            instance = super().__call__(*args, **kwargs)
            cls.instances.append(instance)
            print(len(cls.instances))
            return instance
        print("11")
        return cls.instances[-1] #need to retun instance when 3rd instance created
    
class est(metaclass=Doubleton):
    
    def __init__(self, name):
        self.name = name

a = est("A")
b = est("B")
c = est("C")

print(a.name, b.name, c.name)

in call
1
in call
2
in call
11
A B B


In [84]:
class SingletonMeta(type):
    instances = []
    
    def __call__(cls,*args, **kwargs):
        #print(cls.__name__)
        if any(cls for cls in cls.instances):
            
            #print(cls, len(cls.instances))
            
            return cls.instances
        else:
            #print(len(cls.instances))
            cls.instances.append(super().__call__(*args,**kwargs))
            return cls.instances

class Singleton(metaclass=SingletonMeta):
    pass

a = Singleton()
b = Singleton()
print(a,b)
#print(a.__class__ b.__class__)

[<__main__.Singleton object at 0x000001ADEB702D60>] [<__main__.Singleton object at 0x000001ADEB702D60>]


In [33]:
class OnlyTwoBase(type):
    
    def __new__(cls, name, bases, class_dict):
        if len(bases)<=2 and name == "A"or "D":
            print(f"Creating class {name} with {len(bases)} bases: {bases}")
            return super().__new__(cls,name,bases,class_dict)
        else:
            print(f"Can inherit from only two base classes {bases} and {len(bases)} with name A not {name}")
    
class A(metaclass=OnlyTwoBase):
    def __init__(self):
        print("A")
    
class B(metaclass=OnlyTwoBase):
    def __init__(self):
        print("B")
    
class C(metaclass=OnlyTwoBase):
    def __init__(self):
        print("C")
    
    
class D(A):
    def __init__(self):
        print("Instance of D")
        
    
a = D()
print(a.__class__.__bases__)#get the parent class
print(a.__class__)
print(issubclass(D,A))
print(isinstance(a,D))
print(D.mro())
print(type(D).__mro__) #get the metaclass mro list

Creating class A with 0 bases: ()
Creating class B with 0 bases: ()
Creating class C with 0 bases: ()
Creating class D with 1 bases: (<class '__main__.A'>,)
Instance of D
(<class '__main__.A'>,)
<class '__main__.D'>
True
True
[<class '__main__.D'>, <class '__main__.A'>, <class 'object'>]
(<class '__main__.OnlyTwoBase'>, <class 'type'>, <class 'object'>)


In [10]:
class EnforcingBaseClass(type):
    
    
    def __new__(cls,name,bases,class_dict):
        
        required_base = "MyBase"
        for base in bases + (object,):
            print(base.__name__)
        
        if any(required_base in base.__name__ for base in bases):
            print(f"Created class {name}")
            return super().__new__(cls, name, bases,class_dict)
        else:
            
            print(f"Not allowed to create class {name}")

class MyBase:
    def __init__(self):
        print("MyBase")
    
    
class D(MyBase, metaclass=EnforcingBaseClass):
    def __init__(self):
        print("Instance of D created")

class B(metaclass=EnforcingBaseClass):
    def __init__(self):
        print("B")
        
class A(D): #Indirect inheritance doesn't give __main__.Mybase, instead it gives __main__.D, hence not created 
    def __init__(self):
        print("A")
        
        
d = D()
print(D.mro())
print(d.__class__.__bases__)



MyBase
object
Created class D
object
Not allowed to create class B
D
object
Not allowed to create class A
Instance of D created
[<class '__main__.D'>, <class '__main__.MyBase'>, <class 'object'>]
(<class '__main__.MyBase'>,)


In [57]:
class EnforcingBaseClassMRO(type):
    
    
    def __new__(cls,name,bases,class_dict):
        
        required_base = "MyBase"
        
        new_cls = super().__new__(cls, name, bases,class_dict)
        
        for base in new_cls.__mro__:
            print(base.__name__)
        
        if not any(required_base in base.__name__ for base in new_cls.__mro__):
            print(f"Not allowed to create class {name}")
            
        else: 
            print(f"Created class {name}")
            return new_cls
            
            

class MyBase:
    def __init__(self):
        print("Instance of MyBase")
    
    
class D(MyBase, metaclass=EnforcingBaseClassMRO):
    def __init__(self):
        print("Instance of D")

class B(metaclass=EnforcingBaseClassMRO):
    def __init__(self):
        print("Instance of B")
        
class A(D): #Indirect inheritance MRO chain gives __main__.Mybase, after  __main__.D, hence 
    def __init__(self):
        print("Instance of A")


D
MyBase
object
Created class D
B
object
Not allowed to create class B
A
D
MyBase
object
Created class A


In [54]:
class FixAttributes(type):
    def __new__(cls,name,bases,class_dict):
        
        required_attr = ["name","age"]
        
    
        if all(attrs in class_dict for attrs in required_attr):
            print(f"{class_dict}")
            return super().__new__(cls,name,bases,class_dict)
        else:
            print(f"Missing some attribute, hence can't create class {name}")
                
class Person(metaclass=FixAttributes):
    name = "Default"
    age = 30

class InvalidPerson(metaclass=FixAttributes):
    name = "Missing age"
            

{'__module__': '__main__', '__qualname__': 'Person', 'name': 'Default', 'age': 30}
Missing some attribute, hence can't create class InvalidPerson
